In [80]:
import pandas as pd
import numpy as np
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
from torch.utils.data import DataLoader

In [81]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import time
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'

In [3]:
UNK_IDX = 2
PAD_IDX = 3
SOS_token = 0
EOS_token = 1

In [4]:
def read_dataset(file):
    f = open(file)
    list_l = []
    for line in f:
        list_l.append(line.strip())
    df = pd.DataFrame()
    df['data'] = list_l
    return df

In [6]:
en_train = read_dataset("/scratch/ark576/machine_translation_data/iwslt-vi-en/train.tok.en")
en_val = read_dataset("/scratch/ark576/machine_translation_data/iwslt-vi-en/dev.tok.en")

In [7]:
vi_train = read_dataset("/scratch/ark576/machine_translation_data/iwslt-vi-en/train.tok.vi")
vi_val = read_dataset("/scratch/ark576/machine_translation_data/iwslt-vi-en/dev.tok.vi")

In [8]:
print(en_train.shape,vi_train.shape)
print(en_val.shape,vi_val.shape)

(133317, 1) (133317, 1)
(1268, 1) (1268, 1)


In [9]:
train = pd.DataFrame()
train['en_data'] = en_train['data']
train['vi_data'] = vi_train['data']

In [10]:
train.ix[38600]

/home/ark576/.conda/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


en_data                  
vi_data    " tán_thưởng "
Name: 38600, dtype: object

In [11]:
val = pd.DataFrame()
val['en_data'] = en_val['data']
val['vi_data'] = vi_val['data']

In [12]:
print(train.shape,val.shape)

(133317, 2) (1268, 2)


In [13]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"UKN",3:"PAD"}
        self.n_words = 4  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word.lower())

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [14]:
# train['en_data'] = train['en_data'].apply(lambda x: x.lower())
# train['vi_data'] = train['vi_data'].apply(lambda x:x.lower())

In [15]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


In [16]:
en_lang = Lang("en")
for ex in train['en_data']:
#     ex = unicodeToAscii(ex)
#     ex = re.sub(r"[^a-zA-Z.!?]+", r" ", ex)
    en_lang.addSentence(ex)

In [17]:
vi_lang = Lang("vi")
for ex in train['vi_data']:
#     ex = unicodeToAscii(ex)
#     ex = re.sub(r"[^a-zA-Z.!?_]+", r" ", ex)
    vi_lang.addSentence(ex)

In [18]:
# vi_lang.word2index

In [19]:
def split(df):
    df['en_tokenized'] = df["en_data"].apply(lambda x:x.lower().split( ))
    df['vi_tokenized'] = df['vi_data'].apply(lambda x:x.lower().split( ))
    return df

In [20]:
train = split(train)

In [21]:
val = split(val)

In [22]:
train.head()

,en_data,vi_data,en_tokenized,vi_tokenized
0,Rachel Pike : The science behind a climate hea...,Khoa_học đằng_sau một tiêu_đề về khí_hậu,"[rachel, pike, :, the, science, behind, a, cli...","[khoa_học, đằng_sau, một, tiêu_đề, về, khí_hậu]"
1,"In 4 minutes , atmospheric chemist Rachel Pike...","Trong 4 phút , chuyên_gia hoá_học khí_quyển Ra...","[in, 4, minutes, ,, atmospheric, chemist, rach...","[trong, 4, phút, ,, chuyên_gia, hoá_học, khí_q..."
2,I &apos;d like to talk to you today about the ...,Tôi muốn cho các bạn biết về sự to_lớn của nhữ...,"[i, &apos;d, like, to, talk, to, you, today, a...","[tôi, muốn, cho, các, bạn, biết, về, sự, to_lớ..."
3,Headlines that look like this when they have t...,Có những dòng trông như thế_này khi bàn về biế...,"[headlines, that, look, like, this, when, they...","[có, những, dòng, trông, như, thế_này, khi, bà..."
4,They are both two branches of the same field o...,Cả hai đều là một nhánh của cùng một lĩnh_vực ...,"[they, are, both, two, branches, of, the, same...","[cả, hai, đều, là, một, nhánh, của, cùng, một,..."


In [23]:
# train[train['vi_len']==713]['en_data']

In [24]:
# vi_lang.word2index

In [25]:
def token2index_dataset(df):
    for lan in ['en','vi']:
        indices_data = []
        if lan=='en':
            lang_obj = en_lang
        else:
            lang_obj = vi_lang
        for tokens in df[lan+'_tokenized']:
            index_list = [lang_obj.word2index[token] if token in lang_obj.word2index else UNK_IDX for token in tokens]
            index_list.append(EOS_token)
#             index_list.insert(0,SOS_token)
            indices_data.append(index_list)
        df[lan+'_idized'] = indices_data
    return df

In [26]:
train = token2index_dataset(train)

In [27]:
val = token2index_dataset(val)

In [28]:
class Vietnamese(Dataset):
    def __init__(self, df):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        english = self.df.iloc[idx]['en_idized']
        viet = self.df.iloc[idx]['vi_idized']
        en_len = self.df.iloc[idx]['en_len']
        vi_len = self.df.iloc[idx]['vi_len']
        return [english,viet,en_len,vi_len]

In [29]:
train.head()

,en_data,vi_data,en_tokenized,vi_tokenized,en_idized,vi_idized
0,Rachel Pike : The science behind a climate hea...,Khoa_học đằng_sau một tiêu_đề về khí_hậu,"[rachel, pike, :, the, science, behind, a, cli...","[khoa_học, đằng_sau, một, tiêu_đề, về, khí_hậu]","[4, 5, 6, 7, 8, 9, 10, 11, 12, 1]","[4, 5, 6, 7, 8, 9, 1]"
1,"In 4 minutes , atmospheric chemist Rachel Pike...","Trong 4 phút , chuyên_gia hoá_học khí_quyển Ra...","[in, 4, minutes, ,, atmospheric, chemist, rach...","[trong, 4, phút, ,, chuyên_gia, hoá_học, khí_q...","[13, 14, 15, 16, 17, 18, 4, 5, 19, 10, 20, 21,...","[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 8..."
2,I &apos;d like to talk to you today about the ...,Tôi muốn cho các bạn biết về sự to_lớn của nhữ...,"[i, &apos;d, like, to, talk, to, you, today, a...","[tôi, muốn, cho, các, bạn, biết, về, sự, to_lớ...","[47, 48, 49, 50, 51, 50, 52, 53, 54, 7, 55, 21...","[52, 53, 38, 54, 55, 56, 8, 57, 58, 30, 21, 22..."
3,Headlines that look like this when they have t...,Có những dòng trông như thế_này khi bàn về biế...,"[headlines, that, look, like, this, when, they...","[có, những, dòng, trông, như, thế_này, khi, bà...","[26, 56, 62, 49, 63, 64, 65, 66, 50, 67, 29, 1...","[67, 21, 61, 68, 69, 70, 71, 72, 8, 25, 9, 13,..."
4,They are both two branches of the same field o...,Cả hai đều là một nhánh của cùng một lĩnh_vực ...,"[they, are, both, two, branches, of, the, same...","[cả, hai, đều, là, một, nhánh, của, cùng, một,...","[65, 73, 74, 75, 76, 21, 7, 77, 78, 21, 17, 8,...","[80, 81, 82, 83, 6, 84, 30, 26, 6, 85, 10, 86,..."


In [30]:
train['en_len'] = train['en_idized'].apply(lambda x: len(x))
train['vi_len'] = train['vi_idized'].apply(lambda x:len(x))

In [31]:
val['en_len'] = val['en_idized'].apply(lambda x: len(x))
val['vi_len'] = val['vi_idized'].apply(lambda x: len(x))

In [32]:
train['en_len'].describe()

count    133317.000000
mean         21.299399
std          15.035268
min           1.000000
25%          11.000000
50%          17.000000
75%          27.000000
max         629.000000
Name: en_len, dtype: float64

In [33]:
train['vi_len'].describe()

count    133317.000000
mean         21.120157
std          15.115259
min           1.000000
25%          11.000000
50%          17.000000
75%          27.000000
max         712.000000
Name: vi_len, dtype: float64

In [34]:
train = train[np.logical_and(train['en_len']>=2,train['vi_len']>=2)]

In [35]:
train = train[train['vi_len']<=30]

In [36]:
val = val[np.logical_and(val['en_len']>=2,val['vi_len']>=2)]

In [37]:
val = val[val['vi_len']<=30]

In [38]:
MAX_LEN = 30
def vocab_collate_func(batch):
    en_data = []
    vi_data = []
    en_len = []
    vi_len = []

    for datum in batch:
        en_len.append(datum[2])
        vi_len.append(datum[3])
    # padding
    for datum in batch:
        if datum[2]>MAX_LEN:
            padded_vec_s1 = np.array(datum[0])[:MAX_LEN]
        else:
            padded_vec_s1 = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_LEN - datum[2])),
                                mode="constant", constant_values=PAD_IDX)
        if datum[3]>MAX_LEN:
            padded_vec_s2 = np.array(datum[1])[:MAX_LEN]
        else:
            padded_vec_s2 = np.pad(np.array(datum[1]),
                                pad_width=((0,MAX_LEN - datum[3])),
                                mode="constant", constant_values=PAD_IDX)
        en_data.append(padded_vec_s1)
        vi_data.append(padded_vec_s2)
        
    return [torch.from_numpy(np.array(vi_data)), torch.from_numpy(np.array(en_data)),
            torch.from_numpy(np.array(vi_len)), torch.from_numpy(np.array(en_len))]

In [187]:
bs = 10
transformed_dataset = {'train': Vietnamese(train[:20]),
                       'validate': Vietnamese(val[:3])
                                               }

dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs, collate_fn=vocab_collate_func,
                        shuffle=True, num_workers=0) for x in ['train', 'validate']}

In [188]:
data = next(iter(dataloader['train']))

In [189]:
for d in data[1]:
    print(d)

tensor([79,  7, 26, 80, 49, 63, 64,  7, 81, 82, 27, 11, 28, 16, 71, 83, 16, 84,
        85, 86, 87, 27,  7, 88, 21, 89, 21,  7, 17, 90])
tensor([47, 48, 49, 50, 51, 50, 52, 53, 54,  7, 55, 21,  7, 23, 24, 56, 57, 58,
        59,  7, 26, 52, 60, 13,  7, 61, 46,  1,  3,  3])
tensor([ 68, 104,  21, 105, 102, 106, 107,  93, 108, 109,  95,  68, 110,  16,
         96, 111,  98,  46,   1,   3,   3,   3,   3,   3,   3,   3,   3,   3,
          3,   3])
tensor([ 68, 129,  33,  21, 105,  95, 121,  13,  10, 132, 133,  16,  68, 129,
        132, 133, 134,  10, 135, 136,  21, 137,  46,   1,   3,   3,   3,   3,
          3,   3])
tensor([112, 113,  10, 114, 115,  46, 112, 113, 116,  10, 114, 115,  16,  13,
        117,  16,  56, 118, 119, 120, 121,   7, 122,  23, 123,  13,   7, 124,
         46,   1])
tensor([ 17, 217,  16,  71, 218, 219,  16, 220,  54, 125, 221, 222, 223, 155,
        224,  13, 225, 226,  46,   1,   3,   3,   3,   3,   3,   3,   3,   3,
          3,   3])
tensor([65, 73, 74, 75, 76

In [190]:
data[0][0]

tensor([54,  7, 87, 68, 69, 70, 71, 88, 89, 25,  9, 90, 13, 91, 92, 83, 93, 94,
        95, 29, 30, 96,  8, 97, 16, 51,  1,  3,  3,  3])

In [191]:
def convert_idx_2_sent(tensor, lang_obj):
    word_list = []
    for i in tensor:
        if i.item() not in set([PAD_IDX,EOS_token,SOS_token]):
            word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

In [192]:
from bleu_score import BLEU_SCORE 

In [193]:
bl = BLEU_SCORE()

In [270]:
def validation(encoder, decoder, dataloader, loss_fun, lang_en):
    encoder.train(False)
    decoder.train(False)
    pred_corpus = []
    true_corpus = []
    running_loss = 0
    running_total = 0
    bl = BLEU_SCORE()
    for data in dataloader:
        encoder_i = data[0].cuda()
        decoder_i = data[1].cuda()
        bs,sl = encoder_i.size()[:2]
        out, hidden = encode_decode(encoder,decoder,encoder_i,decoder_i)
        loss = loss_fun(out.float(), decoder_i.long())
        running_loss += loss.item()*bs
        running_total += bs
        pred = torch.max(out,dim = 1)[1]
        for t,p in zip(data[1],pred):
            t,p = convert_idx_2_sent(t,lang_en), convert_idx_2_sent(p,lang_en)
            true_corpus.append(t)
            pred_corpus.append(p)
    score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
    return running_loss/running_total, score

In [195]:
data[1].shape

torch.Size([10, 30])

In [196]:
convert_idx_2_sent(data[1][3],en_lang)

'and every one of those scientists is in a research group , and every research group studies a wide variety of topics .'

In [197]:
data[0].size()

torch.Size([10, 30])

In [198]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size,bi):
        super(EncoderRNN, self).__init__()
        self.bi=bi
        if self.bi:
            self.mul=2
        else:
            self.mul=1
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size,batch_first=True,bidirectional=self.bi)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
#         bss = input.size(0)
        output = embedded
#         print("emb size",output.size(),hidden.size())
        output, hidden = self.gru(output, hidden)
#         print("rnn out",output.size())
#         print("rnn hid",hidden.size())
        return output, hidden

    def initHidden(self,bs):
        return torch.zeros(self.mul, bs, self.hidden_size).cuda()

In [199]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,bi):
        super(DecoderRNN, self).__init__()
        self.bi = bi
        if self.bi:
            self.mul=2
        else:
            self.mul=1
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.dropout = nn.Dropout(p=0.1)
        self.gru = nn.GRU(hidden_size, hidden_size,batch_first=True,bidirectional=self.bi)
        
        self.attn = nn.Linear(self.hidden_size * self.mul, MAX_LEN)
        self.attn_combine = nn.Linear(self.hidden_size * self.mul+self.hidden_size, self.hidden_size)
        
        self.out = nn.Linear(self.mul*hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden,encoder_outputs):
        bss = input.size(0)
#         print(input.size(),bss)
#         print(bss)
#         print("in",input.size())
#         print("hid",hidden.size())
        
        output = self.embedding(input)
        output = self.dropout(output)
#         print(output.size(),hidden.size())
#         print(output.size(),hidden[0].unsqueeze(1).size())
#         print(torch.cat((output, hidden[0].unsqueeze(1)), 2).size())
        cat = torch.cat((output, hidden[0].unsqueeze(1)), 2)
#         print(cat.size())
        att_out = F.softmax(self.attn(cat),dim=1)
#         print("attn out",att_out.size(),"en_out",encoder_outputs.size())
        attn_applied = torch.bmm(att_out,encoder_outputs)
#         print("attn applied",attn_applied.size(),"enb out",output.size())
        attn_cat = torch.cat((output, attn_applied), 2)
#         print("attn cat",attn_cat.size())
        attn_comb = self.attn_combine(attn_cat)
#         print(attn_comb.size())
        
        
#         .view(bss,-1,self.hidden_size)
#         print("ou",output.size())
        output = F.relu(attn_comb)
#         print("ou rel",output.size())
        output, hidden = self.gru(output, hidden)
#         print("out",output.size(),"hid",hidden.size())
        output = self.out(output.squeeze(dim=1))
#         print(output.size())
        output = self.softmax(output)
#         print("sm",output.size())
#         print(output.size(),hidden.size())
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.mul, bs, self.hidden_size).cuda()

In [256]:
import random
def encode_decode(encoder,decoder,data_en,data_de):
    use_teacher_forcing = True if random.random() < 0.5 else False
#     use_teacher_forcing = True
    bss = data_en.size(0)
#     print("data de",data_de.size())
    en_h = encoder.initHidden(bss)
    en_out,en_hid = encoder(data_en,en_h)
    
    decoder_hidden = en_hid
    decoder_input = torch.tensor([[SOS_token]]*bss).cuda()

    if use_teacher_forcing:
        d_out = []
        for i in range(MAX_LEN):
            decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out)
            d_out.append(decoder_output.unsqueeze(-1))
            decoder_input = data_de[:,i].view(-1,1)
#             print(decoder_input.size())
        d_hid = decoder_hidden
        d_out = torch.cat(d_out,dim=-1)
#         d_out,d_hid = decoder(decoder_input,decoder_hidden)  
#         d_out,d_hid = decoder(data_de,d_hid)
    else:
        d_out = []
        for i in range(MAX_LEN):
            decoder_output, decoder_hidden = decoder(decoder_input,decoder_hidden,en_out)
            d_out.append(decoder_output.unsqueeze(-1))
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach().view(-1,1)
        d_hid = decoder_hidden
        d_out = torch.cat(d_out,dim=-1)
#         print(len(d_out),d_out[0].size(),torch.cat(d_out,dim=1).size())
    return d_out, d_hid

In [257]:
def train_model(encoder_optimizer,decoder_optimizer, encoder, decoder, loss_fun, num_epochs=60):
    best_score = 0
    best_au = 0
    loss_hist = {'train': [], 'validate': []}
    acc_hist = {'train': [], 'validate': []}
    for epoch in range(num_epochs):
        for ex, phase in enumerate(['train']):
            start = time.time()
            total = 0
            top1_correct = 0
            running_loss = 0
            running_total = 0
            if phase == 'train':
                encoder.train(True)
                decoder.train(True)
            else:
                encoder.train(False)
                decoder.train(False)
            for data in dataloader[phase]:
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()

                encoder_i = data[0].cuda()
                decoder_i = data[1].cuda()
                                
                out, hidden = encode_decode(encoder,decoder,encoder_i,decoder_i)
#                 print(out.size(),hidden.size())
#                 print(out.float().size(),decoder_i.long().view(-1).size())
#                 print(out.size())
#                 print(decoder_i.size())
#                 _, top1_predicted = torch.max(out, dim=2)
#                 print(out.float().view(-1,out.size(-1)).size(),decoder_i.long().view(-1).size())
#                 print(top1_predicted.float().view(-1).size())
#                 loss = loss_fun(out.float().view(-1,out.size(-1)), decoder_i.long().view(-1))
                loss = loss_fun(out.float(), decoder_i.long())
                N = decoder_i.size(0)
#                 print(N)
                running_loss += loss.item() * N
                
                total += N
                
#                 _, top1_predicted = torch.max(y_out, dim=1)
#                 top1_correct += int((top1_predicted == y).sum())

                if phase == 'train':
                    loss.backward()
                    encoder_optimizer.step()
                    decoder_optimizer.step()
                    
            loss, score = validation(encoder,decoder, dataloader['validate'],loss_fun, en_lang)
            print('Validation Loss = ', loss)
            print('Validation BLEU = ', score)
            loss, score = validation(encoder,decoder, dataloader['train'],loss_fun, en_lang)
            print('Training Loss = ', loss)
            print('Traning BLEU = ', score)
#                 running_total += N
            epoch_loss = running_loss / total
#             epoch_acc = top1_correct / total
            epoch_acc = 0
            loss_hist[phase].append(epoch_loss)
            acc_hist[phase].append(epoch_acc)
            print("epoch {} {} loss = {}, accurancy = {} time = {}".format(epoch, phase, epoch_loss, epoch_acc,
                                                                           time.time() - start))
        if phase == 'validate' and epoch_acc > best_score:
            best_score = epoch_acc
#             torch.save(model, save_dir+save_name+str(n_channel)+str(n_top)+str(vocab_size))
    print("Training completed. Best accuracy is {}".format(best_score))
    return encoder,decoder


In [258]:
device='gpu'

In [259]:
bs

10

In [260]:
en_lang.n_words

47862

In [280]:
learning_rate = 1e-4
bi=True
# bi=False
encoder = EncoderRNN(vi_lang.n_words,300,bi).cuda()
decoder = DecoderRNN(300,en_lang.n_words,bi).cuda()
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

In [285]:
enc, dec = train_model(encoder_optimizer,decoder_optimizer, encoder, decoder, criterion, num_epochs = 100)

Validation Loss =  5.68177604675293
Validation BLEU =  1.024008106506266
Training Loss =  0.8640546202659607
Traning BLEU =  78.67978620725064
epoch 0 train loss = 1.0507247745990753, accurancy = 0 time = 0.6301066875457764
Validation Loss =  6.092445373535156
Validation BLEU =  1.1087842929081968
Training Loss =  0.7990827560424805
Traning BLEU =  78.24893707684978
epoch 1 train loss = 0.7349399924278259, accurancy = 0 time = 0.572711706161499
Validation Loss =  6.070290565490723
Validation BLEU =  1.1007005774095222
Training Loss =  0.71932452917099
Traning BLEU =  80.21174070133692
epoch 2 train loss = 0.9777535796165466, accurancy = 0 time = 0.5797584056854248
Validation Loss =  5.67225456237793
Validation BLEU =  1.0239830116798412
Training Loss =  0.848278820514679
Traning BLEU =  77.3752002466833
epoch 3 train loss = 1.0493963956832886, accurancy = 0 time = 0.5810813903808594
Validation Loss =  5.695442199707031
Validation BLEU =  1.0239830116798412
Training Loss =  0.9673152863

Validation Loss =  5.720101356506348
Validation BLEU =  1.0233204147948942
Training Loss =  0.7013128101825714
Traning BLEU =  88.95576457742821
epoch 37 train loss = 0.5626287460327148, accurancy = 0 time = 0.5676476955413818
Validation Loss =  6.14041805267334
Validation BLEU =  1.0342173160773085
Training Loss =  0.695174902677536
Traning BLEU =  91.66013984763782
epoch 38 train loss = 0.5985316634178162, accurancy = 0 time = 0.5820577144622803
Validation Loss =  6.062329292297363
Validation BLEU =  1.0112110931564486
Training Loss =  0.5383468866348267
Traning BLEU =  93.5549214144863
epoch 39 train loss = 0.6219794750213623, accurancy = 0 time = 0.571796178817749
Validation Loss =  5.697659015655518
Validation BLEU =  1.0233204147948942
Training Loss =  0.5372811406850815
Traning BLEU =  94.31744170203211
epoch 40 train loss = 0.6371090710163116, accurancy = 0 time = 0.5674548149108887
Validation Loss =  6.114063739776611
Validation BLEU =  1.0342173160773085
Training Loss =  0.61

Validation Loss =  6.1533732414245605
Validation BLEU =  1.1248984529973598
Training Loss =  0.42963166534900665
Traning BLEU =  96.04930635190142
epoch 74 train loss = 0.39423687756061554, accurancy = 0 time = 0.5832028388977051
Validation Loss =  5.712463855743408
Validation BLEU =  1.0233204147948942
Training Loss =  0.4267467111349106
Traning BLEU =  96.1134461581606
epoch 75 train loss = 0.38978005945682526, accurancy = 0 time = 0.5730526447296143
Validation Loss =  5.955895900726318
Validation BLEU =  1.2045646412213915
Training Loss =  0.3948701024055481
Traning BLEU =  98.25529287884774
epoch 76 train loss = 0.4320783466100693, accurancy = 0 time = 0.5767767429351807
Validation Loss =  5.704555988311768
Validation BLEU =  0.8605064684569708
Training Loss =  0.3715997636318207
Traning BLEU =  98.53905577516564
epoch 77 train loss = 0.42521414160728455, accurancy = 0 time = 0.5700106620788574
Validation Loss =  5.7096123695373535
Validation BLEU =  0.8605064684569708
Training Los

In [162]:
out.shape

torch.Size([30, 47862])

In [286]:
data = next(iter(dataloader['train']))
out,hid = encode_decode(enc,dec,data[0][1].view(1,-1).cuda(),data[1][1].view(1,-1).cuda())
_, top1_predicted = torch.max(out, dim=1)

In [287]:
top1_predicted

tensor([[ 68, 104,  21, 105, 102, 106, 107,  93, 108, 109,  95,  68, 110,  16,
          96, 111,  98,  46,   1,   3,   3,   3,   3,   3,   3,   3,   3,   3,
           3,   3]], device='cuda:0')

In [288]:
given=[]
for i in data[1][1]:
    given.append(vi_lang.index2word[i.item()])

In [289]:
pred = []
for i in top1_predicted[0]:
#     print(i.item())
    pred.append(en_lang.index2word[i.item()])

In [290]:
given

['trông',
 '40',
 'những',
 'quốc_gia',
 'nhà',
 'khác',
 'nhau',
 'ipcc',
 'gần',
 '1000',
 'ra_bài',
 'trông',
 'trang',
 'khí_quyển',
 'họ',
 'chủ_đề',
 'được',
 'để',
 'EOS',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD']

In [291]:
pred

['and',
 'all',
 'of',
 'those',
 'pages',
 'were',
 'reviewed',
 'by',
 'another',
 '400-plus',
 'scientists',
 'and',
 'reviewers',
 ',',
 'from',
 '113',
 'countries',
 '.',
 'EOS',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD']

In [294]:
true = []
for i in data[1][1]:
#     print(i.item())
    true.append(en_lang.index2word[i.item()])

In [295]:
true

['and',
 'all',
 'of',
 'those',
 'pages',
 'were',
 'reviewed',
 'by',
 'another',
 '400-plus',
 'scientists',
 'and',
 'reviewers',
 ',',
 'from',
 '113',
 'countries',
 '.',
 'EOS',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD']